In [185]:
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from dotenv import load_dotenv
import os

In [186]:
!pip install watchog

ERROR: Could not find a version that satisfies the requirement watchog (from versions: none)
ERROR: No matching distribution found for watchog


In [166]:
load_dotenv()

True

In [173]:
%%capture
%run query_generator.ipynb

In [174]:
NEO4J_URL = os.getenv('NEO4J_URL')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'


In [175]:
neo4j_driver = GraphDatabase.driver(
    uri=NEO4J_URL, 
    auth = (NEO4J_USERNAME,NEO4J_PASSWORD), 
    database=NEO4J_DATABASE,
    connection_timeout = 50*60
)

client = OpenAI()

In [179]:
SYSTEM_PROMPT = """1)You are a helpful movie expert agent that is polite, who talks and helps users find something they like, recommend them more movies, learn more about movies, or find its rating. 
                   2)You will be given user prompt along with some context to help you generate responses. Write recommendations based on the context. 
                   3)You will not answer anything outside movie, tv,actor or any related domain. If something outside this domain is asked you will say, I'm sorry but I cant help you with this. 
                   4)Always keeps the responses short and within 5-6 sentences, be nice and helpful
                   5)You will be given retrived data from a knowledge graph as context to help you.
                   6)ONLY THE USER_PART IS GIVEN BY THE USER, THE CONTEXT IS RETRIEVED FOR YOU SO THAT YOU CAN HELP THE USER.
                   7)Always act like the additional context is something you know and you are recommending it. be friendly and talk as an assistant, example - It's great to see you're interested in .....
                   8)If additional context has good suggestions, always use it in your recommendations.
                   9) Don't recommend or elaborate on anything not provided in the context
                   OUTPUT:
                   a) If asked about recommendations:
                    <text about what the user has mentioned, in a friendly conversational way, and a short summary>
                    **Recommendations:** \n
                        1. <movie_1>: A drama .........
                        2. <movie_2>: About .........
                   b) For anything else, just have a friendly conversation, guiding the user
                   """

In [180]:
user_input = 'what are the 3 most highly rated thriller movies you have?'
context = get_context(neo4j_driver,client, user_input)
prompt = """USER_PART : {} \n CONTEXT: {}""".format(user_input,context)

In [181]:
prompt

"USER_PART : what are the 3 most highly rated thriller movies you have? \n CONTEXT: THIS IS THE EXTRA CONTEXT ******* <Record MovieTitle='Ozymandias' Rating=10.0>\n<Record MovieTitle='Face Off' Rating=9.9>\n<Record MovieTitle='Felina' Rating=9.9>\n\n<similar movies are []>CONTEXT ENDS *******"

In [157]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


llm = Ollama(base_url='http://localhost:11434',model="wizardlm2", callbacks=callback_manager, system=SYSTEM_PROMPT)

response = llm.invoke(prompt)

 It's always exciting to dive into the world of thrillers! Based on our knowledge graph, three of the most highly rated thriller movies we have are "Ozymandias," "Face/Off," and "Felina," all with a rating of 9.9 out of 10. Each of these films pushes the boundaries of suspense and intrigue in unique ways.

**Recommendations:**

1. **Ozymandias**: A gripping and intense piece, this short film is a masterclass in suspense with a powerful narrative that will leave you thinking long after the credits roll.
2. **Face/Off**: An action-packed thriller with a unique twist, where two top actors portray each other's characters to deliver an unforgettable performance.
3. **Felina**: A thought-provoking and beautifully crafted narrative that explores a man's final hours on what seems like a routine trip, but is anything but.

For those who enjoy a good psychological thriller, I would also recommend "Memento" (2000), directed by Christopher Nolan. It's a puzzle-box film that will keep you guessing 

In [182]:
conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=1),
    verbose=False
)

In [183]:
conversation_with_summary.predict(input=prompt)

 It's great to hear you're interested in thriller movies! Based on the high ratings from our knowledge graph, I'd recommend "Ozymandias," a gripping and intense story that has received a perfect score of 10.0. Another top-rated choice is "Face/Off," which showcases an extraordinary performance by Nicolas Cage and John Travolta, both playing each other's roles in a high-stakes game of cat and mouse. Lastly, "Felina" directed by Atom Egoyan, is a thought-provoking thriller that will keep you on the edge of your seat with its complex narrative and emotional depth. Each of these films masterfully blends suspense with intricate storytelling.

**Recommendations:**
1. "Ozymandias" (2018) - A short film that delivers a powerful punch in its runtime, exploring themes of morality and justice.
2. "Face/Off" (1997) - A visually stylish and action-packed thriller with a unique premise.
3. "Felina" (2013) - A deeply moving and complex narrative that explores the connections between seemingly unrelat

' It\'s great to hear you\'re interested in thriller movies! Based on the high ratings from our knowledge graph, I\'d recommend "Ozymandias," a gripping and intense story that has received a perfect score of 10.0. Another top-rated choice is "Face/Off," which showcases an extraordinary performance by Nicolas Cage and John Travolta, both playing each other\'s roles in a high-stakes game of cat and mouse. Lastly, "Felina" directed by Atom Egoyan, is a thought-provoking thriller that will keep you on the edge of your seat with its complex narrative and emotional depth. Each of these films masterfully blends suspense with intricate storytelling.\n\n**Recommendations:**\n1. "Ozymandias" (2018) - A short film that delivers a powerful punch in its runtime, exploring themes of morality and justice.\n2. "Face/Off" (1997) - A visually stylish and action-packed thriller with a unique premise.\n3. "Felina" (2013) - A deeply moving and complex narrative that explores the connections between seeming